In [ ]:
import pprint
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import copy
import re
from time import sleep
from selenium import webdriver

from pymongo import MongoClient

The following function scrapes the fees and home descriptions from Airbnb given url 

In [ ]:
def web_scraping(df,db):
    for i, url in enumerate(df.listing_url):
        day = df.minimum_nights[i]+1
        if db.count_documents({'link':url})==0:
            try:
                if day<10:
                    new_url = url + f'?source_impression_id=p3_1616194878_hGlc%2FRxTeggNQPIp&check_in=2021-07-01&guests=1&adults=1&check_out=2021-07-0{day}'
                else:
                    new_url = url + f'?source_impression_id=p3_1616194878_hGlc%2FRxTeggNQPIp&check_in=2021-07-01&guests=1&adults=1&check_out=2021-07-{day}'
                    
                driver = webdriver.Firefox()
                driver.get(new_url)
                time.sleep(6)
                htmlSource = driver.page_source
                soup = BeautifulSoup(htmlSource,'html.parser')
            
                cleanliness = 0
                Accuracy = 0
                Communication = 0
                Location = 0
                Checkin = 0
                Value = 0
                Final_score = 0
                Description = 'None'
            
                Description = soup.find_all('div','_1d784e5')[0].text 
                cleanliness = soup.find_all('div',"_a3qxec")[0].find('span','_4oybiu').text
                Accuracy = soup.find_all('div',"_a3qxec")[1].find('span','_4oybiu').text
                Communication = soup.find_all('div',"_a3qxec")[2].find('span','_4oybiu').text
                Location = soup.find_all('div',"_a3qxec")[3].find('span','_4oybiu').text
                Checkin = soup.find_all('div',"_a3qxec")[4].find('span','_4oybiu').text
                Value = soup.find_all('div',"_a3qxec")[5].find('span','_4oybiu').text
                Final_score = soup.find('div',"_544wcx").find('a','_5twioja').text
                
            
                cleaning = soup.find('div',"_ud8a1c").find_all('span','_ra05uc')[1].text
                service = soup.find('div',"_ud8a1c").find_all('span','_ra05uc')[2].text
                
                
            except Exception as e:
            
                cleaning = np.nan
                service = np.nan
             
            record_to_add = {'link':url,'new_link':new_url,'cleaning_fee':cleaning,'service_fee':service, 'cleanliness':cleanliness , 'Accuracy':Accuracy , 'Communication':Communication , 'Location': Location , 'Checkin' : Checkin , 'Value' : Value , 'Final_score':Final_score , 'Description':Description}
            db.insert_one(record_to_add)
            driver.close()
            sleep(1)
            
        else:
            continue
    return db

In [ ]:
df = pd.read_csv("../Data/listings_details.csv")

In [ ]:
client = MongoClient('localhost', 27017)
db = client['airbnb']
comments = db['airbnb_scrape']
client.list_database_names()

In [ ]:
# Scraping the airbnb scores and descriptions
web_scraping(df,comments)

In [ ]:
# Saving the scraped data into a dataframe
scraped_data = pd.DataFrame(list(comments.find({})))

In [ ]:
# Saving the scraped data into a csv file
scraped_data.to_csv('Airbnb_Description.csv')

In [ ]:
# In case to remove the database
# client.drop_database('airbnb')